In [1]:
#https://grouplens.org/datasets/movielens/
import pandas as pd
from pyspark.sql.functions import col, explode
from pyspark import SparkContext

In [2]:
from pyspark.sql import SparkSession
sc = SparkContext
# sc.setCheckpointDir('checkpoint')
spark = SparkSession.builder.config("spark.driver.memory", "4g")
spark = spark.appName('Recommendations').getOrCreate()

22/04/11 23:52:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/11 23:52:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
movies = spark.read.csv("data/movies.csv",header=True)
ratings = spark.read.csv("data/ratings.csv",header=True)

In [4]:
def GenresCombinations(moviesDataset, ratingsDataset):

    # Get a genres list.
    firstGenresList = moviesDataset \
        .withColumn("genres", split(col("genres"), "\\|")) \
        .select(col("movieId"), explode(col("genres"))) \
        .withColumnRenamed("col", "genre")

    # Get a second list of genres with different column names for same-table join.
    secondGenresList = firstGenresList \
        .withColumnRenamed("genre", "r_genre") \
        .withColumnRenamed("movieId", "r_movieId")

    # Join both genre lists to get every combination. Group by both genres in the combination. Count repetitions and filter the greatest for each one.
    greatestCombinationByGenre = firstGenresList \
        .join(secondGenresList, firstGenresList.genre != secondGenresList.r_genre) \
        .where(firstGenresList.movieId == secondGenresList.r_movieId) \
        .groupBy("genre", "r_genre") \
        .count() \
        .orderBy(col("count").desc()) \
        .groupBy("genre") \
        .agg(first("r_genre").alias("most related genre"), max("count").alias("times related")) \
        .orderBy("genre") \
        .select("times related", "genre", "most related genre")

    return greatestCombinationByGenre

In [ ]:
def BestFilmsByRating(moviesDataset, ratingsDataset):


In [7]:
qulifyingMovies = ratings.groupBy("movieId").agg(col("avg(rating)"), col("count(rating)")).filter("count(rating) >= 5000")

AnalysisException: cannot resolve '`avg(rating)`' given input columns: [movieId, rating, timestamp, userId];
'Aggregate [movieId#39], [movieId#39, 'avg(rating), 'count(rating)]
+- Relation[userId#38,movieId#39,rating#40,timestamp#41] csv


In [4]:
ratings = ratings.\
    withColumn('userId', col('userId').cast('integer')).\
    withColumn('movieId', col('movieId').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    296|   5.0|
|     1|    306|   3.5|
|     1|    307|   5.0|
|     1|    665|   5.0|
|     1|    899|   3.5|
|     1|   1088|   4.0|
|     1|   1175|   3.5|
|     1|   1217|   3.5|
|     1|   1237|   5.0|
|     1|   1250|   4.0|
|     1|   1260|   3.5|
|     1|   1653|   4.0|
|     1|   2011|   2.5|
|     1|   2012|   2.5|
|     1|   2068|   2.5|
|     1|   2161|   3.5|
|     1|   2351|   4.5|
|     1|   2573|   4.0|
|     1|   2632|   5.0|
|     1|   2692|   5.0|
+------+-------+------+
only showing top 20 rows



In [5]:
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  99.74% empty.


In [6]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [7]:
# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \


# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  16


In [9]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

CrossValidator_12c68a32263b


In [10]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel

22/04/11 23:45:28 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/04/11 23:45:28 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


KeyboardInterrupt: 

In [ ]:
# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

In [ ]:
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

In [ ]:
# Generate n Recommendations for all users
nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

In [ ]:
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

nrecommendations.limit(10).show()